# Practica 1 Bloque 2
### Predicción ingredientes de la semana Pizzería Maven Pizzas
Lara Ocón Madrid - 202115710


En esta práctica vamos a tomar los datos de la pizzería Maven Pizzas y calcular para cada semana del año la cantidad de cada ingrediente que se ha necesitado. 

Una vez obtenida la cantidad de cada ingrediente para cada semana, calcularemos la media de cada uno de ellos y multiplicaremos dicha media por 1.5 (valor que he decidido tomar para que no haya falta de ingredientes una semana).

Es decir, emplearemos una ETL, extrayendo los datos, transfromándolos para quedarnos con lo que necesitamos cada semana, y cargando dicha predicción en un csv.

In [7]:
# Cargamos primero las librerías necesarias
import pandas as pd
import datetime
import re
import numpy as np

En primer lugar extraemos los datos:

1) EXTRACCIÓN DE DATOS

In [2]:
def extract(fichero):
    df = pd.read_csv(fichero)
    return df

In [3]:
# creamos una función que devuelva un diccionario con los
# ingredientes de cada pizza
def extraer_ingredientes(df):
    # me guardo en un diccionario los ingredientes para cada pizza
    dic = {}
    for i in range(len(df)):
        pizza = df.iloc[i]
        clave = pizza['pizza_type_id']
        ingredientes = pizza['ingredients'].strip().split(", ")
        dic[clave] = ingredientes
    return dic

In [11]:
# vamos a crear una función que tome el df de las orders
# con sus fechas correspondientes y transforme las fechas
# a su formato correspondiente

def transformar_fechas(df):
    # creo una nueva columna con el formato de fecha
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    return df

In [9]:
# creamos una función que devuelva una lista que contenga
# para cada semana su rango de orders. Es decir, las orders 
# están enumeradas por un identificador, el cual va en orden
# ascendente. Por lo que solo con el limite inferior y superior
# de cada semana podemos saber que orders corresponden a cada semana

def extraer_rango_orders_semana(df_order_dates):
    # buscamos el primer martes
    # el primer dia del dataframe es:

    orders_semanas = [[np.inf, -np.inf] for t in range(53)]

    # veo que dia de la semana es el 1 de enero de 2016
    primer_lunes = pd.to_datetime("01-01-2016").dayofweek

    
    # print(df_order_dates.loc[i, 'date'].day_of_year)
    
    i = 0
    while i < len(df_order_dates):
        orders_semana = orders_semanas[(df_order_dates.loc[i, 'date'].day_of_year + primer_lunes) // 7]
        if (df_order_dates.loc[i, 'order_id']) > orders_semana[1]:
            orders_semana[1] = df_order_dates.loc[i, 'order_id']
        if (df_order_dates.loc[i, 'order_id']) < orders_semana[0]:
            orders_semana[0] = df_order_dates.loc[i, 'order_id']
        
        orders_semanas[(df_order_dates.loc[i, 'date'].day_of_year + primer_lunes) // 7] = orders_semana
        
        i +=1 

    print(orders_semanas)
    return orders_semanas


In [37]:
# Extraemos el df con los tipos de pizzas y nos guardamos el nombre de las
# pizzas.

df_pizza_types = extract('pizza_types.csv')

pizzas_id = df_pizza_types['pizza_type_id']


In [20]:

df_orders_fechas = extract('orders.csv')
df_orders_fechas = transformar_fechas(df_orders_fechas)
df_orders_fechas.head()

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30


In [22]:
orders_por_semana = extraer_rango_orders_semana(df_orders_fechas)
print(len(orders_por_semana))

[[1, 136], [137, 564], [565, 975], [976, 1389], [1390, 1791], [1792, 2240], [2241, 2665], [2666, 3077], [3078, 3471], [3472, 3879], [3880, 4298], [4299, 4726], [4727, 5132], [5133, 5563], [5564, 5971], [5972, 6389], [6390, 6820], [6821, 7244], [7245, 7649], [7650, 8108], [8109, 8514], [8515, 8913], [8914, 9329], [9330, 9751], [9752, 10174], [10175, 10573], [10574, 11017], [11018, 11469], [11470, 11893], [11894, 12319], [12320, 12730], [12731, 13154], [13155, 13591], [13592, 14012], [14013, 14413], [14414, 14805], [14806, 15230], [15231, 15662], [15663, 15940], [15941, 16377], [16378, 16721], [16722, 17110], [17111, 17458], [17459, 17809], [17810, 18218], [18219, 18624], [18625, 19015], [19016, 19516], [19517, 19918], [19919, 20343], [20344, 20769], [20770, 21106], [21107, 21350]]
53


In [23]:
detalles_orders = extract('order_details.csv')
detalles_orders.head()

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1


Ya tenemos el rango de id de orders y hemos cargado la informacion de cada order. Vamos a crear un dataframe que nos diga cuantas pizzas de cada tipo se piden cada semana.

In [33]:
# Creamos una función que nos devuelva un df con el numero de 
# pizzas de cada tipo que se han pedido en cada semana

def pizzas_por_semana(orders_semanas, df_order_details, pizzas_id):
    # Esta función recibe la lista que contiene los rangos de las orders
    # por semana (orders_semanas), el df con los detalles de las orders para 
    # cada semana (df_order_details) y una lista con todas las pizzas.

    df_pizzas_semana = pd.DataFrame()               # creamos el dataframe
    datos = {}
    for i in range(53):                             # creamos los indices del dataframe
        datos[f'semana {i}'] = [0 for i in range(len(pizzas_id))] # inicializamos la cantidad de cada pizza en 0
    df_pizzas_semana = pd.DataFrame(datos, index=pizzas_id) 

    i = 0
    semana = 0                                      # empezamos en la semana 0
    while semana < len(orders_semanas) and i < len(df_order_details):
        # buscamos la primera order de la semana correspondiente
        while (i < len(df_order_details)) and (df_order_details.loc[i, 'order_id'] < orders_semanas[semana][0]):
            i += 1
        # en el momento que lo encontramos, empezamos a añadir las pizzas hasta
        # salir del rango de orders de esa semana
        while (i < len(df_order_details)) and (df_order_details.loc[i, 'order_id'] <= orders_semanas[semana][1]):
            pizza, cantidad = obtener_nombre_y_can_pizza(df_order_details.iloc[i])
            df_pizzas_semana.loc[pizza, f'semana {semana}'] += cantidad
            i += 1
        semana += 1                         # hemos terminado, pasamos a la siguiente semana
    
    return df_pizzas_semana


# Dado que en order details, además del id de pizza que se pide tenemos el tamaño
# de dicha pizza, y la cantidad de pizzas que se piden, vamos a crear una función
# que interprete para cada tamaño de pizza, cuantas pizzas se corresponden a un 
# tamaño de pizza "normal" y lo multiplique por la cantidad de dicha pizza.
# Esto nos servirá a la hora de calcular los ingredientes necesarios puesto que 
# no es lo mismo los ingredientes necesarios para una pizza xxl que una m.
# Como los tamaños van de s a xxl usaremos la siguiente correspondencia:
# s = 1, m = 1.5, l = 2, xl = 2.5, xxl = 3

def obtener_nombre_y_can_pizza(order):
    # añado la pizza asociada a ese order
    pizza = order['pizza_id']
    tam = 3
    # vemos su tamaño
    if re.search("_s$", pizza):
        pizza = re.sub("_s$","", pizza)
        tam = 1
    elif re.search("_m$", pizza):
        pizza = re.sub("_m$","", pizza)
        tam = 1.5
    elif re.search("_l$", pizza):
        pizza = re.sub("_l$","", pizza)
        tam = 2
    elif re.search("_xl$", pizza):
        pizza = re.sub("_xl$","", pizza)
        tam = 2.5
    elif re.search("_xxl$", pizza):
        pizza = re.sub("_xxl$","", pizza)
        tam = 3
    
    cantidad = order['quantity']
    
    return pizza, cantidad*tam

In [34]:
df_pizzas_semana = pizzas_por_semana(orders_por_semana, detalles_orders, pizzas_id)
df_pizzas_semana.head()

,semana 0,semana 1,semana 2,semana 3,semana 4,semana 5,semana 6,semana 7,semana 8,semana 9,...,semana 43,semana 44,semana 45,semana 46,semana 47,semana 48,semana 49,semana 50,semana 51,semana 52
pizza_type_id,,,,,,,,,,,,,,,,,,,,,
bbq_ckn,33.0,64.0,74.0,81.0,87.5,84.0,84.5,70.0,68.0,77.5,...,43.0,74.0,75.0,87.5,87.5,80.0,68.0,65.5,62.0,49.5
cali_ckn,15.5,76.0,58.5,90.5,60.0,87.0,73.0,84.0,80.5,86.0,...,58.5,86.5,92.0,60.5,87.0,73.5,78.5,89.0,60.5,47.5
ckn_alfredo,4.0,53.5,18.5,23.5,28.5,32.5,33.0,24.5,24.0,36.0,...,28.5,39.5,47.0,26.5,26.5,29.5,32.5,27.5,27.0,14.0
ckn_pesto,10.5,29.5,29.0,27.5,21.5,22.5,33.5,45.0,28.0,18.0,...,20.0,34.0,30.0,37.5,23.0,40.5,21.0,30.5,25.5,18.0
southw_ckn,20.0,40.0,78.5,43.5,44.5,61.0,56.0,80.0,55.0,82.5,...,59.0,58.5,70.5,59.0,70.5,54.0,81.5,49.0,46.0,34.0


Ya tenemos la cantidad aproximada de pizzas que se ha obtenido para cada semana del año.
Ahora solo tenemos que multiplicar para cada semana y para cada pizza, la cantidad de dicha pizza por los ingredientes necesarios para hacerla. Esta cantidad necesaria de cada
ingrediente la vamos guardando en un dataframe que nos diga para cada semana, cuanto
se ha necesitado de cada ingrediente.

In [38]:
# Vamos a crear un diccionario con los ingredientes necesarios para (un tamaño normal) 
# de cada pizza. Esto lo hacemos puesto que es más rápido recorrer un diccionario.
df_ingredientes = extract('pizza_types.csv')
dic_ingredientes = extraer_ingredientes(df_ingredientes)

In [40]:
# Inicializamos el dataframe que contendrá los ingredientes necesarios por semana
total_ingredientes = []
datos = {}
for ingredientes in  list(dic_ingredientes.values()) :
    for ingrediente in ingredientes:
        if ingrediente not in total_ingredientes:
            total_ingredientes.append(ingrediente)
            datos[ingrediente] = [0 for i in range(53)]

semanas = [f"semana {i}" for i in range(53)]

# ahora creamos un dataframe con los ingredientes de cada semana
df_ingredientes_semanas = pd.DataFrame(datos, index=semanas)
df_ingredientes_semanas.head()

,Barbecued Chicken,Red Peppers,Green Peppers,Tomatoes,Red Onions,Barbecue Sauce,Chicken,Artichoke,Spinach,Garlic,...,Smoked Gouda Cheese,Romano Cheese,Blue Cheese,Ricotta Cheese,Gorgonzola Piccante Cheese,Parmigiano Reggiano Cheese,Eggplant,Zucchini,Sun-dried Tomatoes,Plum Tomatoes
semana 0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
semana 1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
semana 2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
semana 3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
semana 4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
# Creamos una función ºque nos devuelva un df con los ingredientes necesarios por semana

def extraer_ingredientes_semanas(df_ingredientes_semanas, df_pizzas_semana, dic_ingredientes):
    # Vamos recorriendo las pizzas, y multiplicando para cada semana, la cantidad de dicha
    # pizza por los ingredientes que necesita, y lo vamos sumando al dataframe de ingredientes
    
    for pizza in df_pizzas_semana.index:
        
        ingredientes = dic_ingredientes[pizza]

        for j in range(len(df_ingredientes_semanas)):
            # voy recorriendo las filas del df de ingredientes (semanas) y obtengo cuantas pizzas
            # de ese tipo se hacen cada semana
            num = df_pizzas_semana.loc[pizza, f"semana {j}"]
            # añado el numero a cada ingrediente
            for ingrediente in ingredientes:
                df_ingredientes_semanas.loc[f"semana {j}", ingrediente] += num
            
    return df_ingredientes_semanas

In [48]:
df_ingredientes_semanas = extraer_ingredientes_semanas(df_ingredientes_semanas, df_pizzas_semana, dic_ingredientes)
df_ingredientes_semanas.head()

,Barbecued Chicken,Red Peppers,Green Peppers,Tomatoes,Red Onions,Barbecue Sauce,Chicken,Artichoke,Spinach,Garlic,...,Smoked Gouda Cheese,Romano Cheese,Blue Cheese,Ricotta Cheese,Gorgonzola Piccante Cheese,Parmigiano Reggiano Cheese,Eggplant,Zucchini,Sun-dried Tomatoes,Plum Tomatoes
semana 0,99.0,574.5,168.0,973.5,669.0,99.0,237.0,46.5,268.5,931.5,...,78,78,78,48.0,48.0,48.0,22.5,63.0,39.0,21.0
semana 1,192.0,1504.5,406.5,2452.5,1765.5,192.0,853.5,228.0,928.5,2671.5,...,204,204,204,228.0,228.0,228.0,108.0,244.5,159.0,75.0
semana 2,222.0,1482.0,486.0,2643.0,1915.5,222.0,750.0,175.5,874.5,2691.0,...,180,180,180,160.5,160.5,160.5,73.5,234.0,168.0,70.5
semana 3,243.0,1446.0,513.0,2386.5,1669.5,243.0,799.5,271.5,883.5,2641.5,...,204,204,204,217.5,217.5,217.5,120.0,261.0,144.0,79.5
semana 4,262.5,1456.5,466.5,2361.0,1731.0,262.5,675.0,180.0,732.0,2443.5,...,144,144,144,169.5,169.5,169.5,102.0,214.5,154.5,75.0


Ya tenemos toda la información que necesitamos. Solo nos queda calcular la predicción. 
Para ello, vamos a tomar la media que se necesita de cada ingrediente y multiplicar dicha
cantida por 1.5 (para evitar que haya escasez de dicho ingrediente una semana)

In [46]:
def obtener_prediccion_ingredientes(df_ingredientes_semanas):
    predicciones = df_ingredientes_semanas.mean()*1.5
    df_prediccion = pd.DataFrame(data=predicciones, columns=['cantidad'])
    return df_prediccion

df_prediccion = obtener_prediccion_ingredientes(df_ingredientes_semanas)
df_prediccion.head(len(df_prediccion))

,cantidad
Barbecued Chicken,220.867925
Red Peppers,1489.330189
Green Peppers,459.283019
Tomatoes,2447.688679
Red Onions,1783.726415
...,...
Parmigiano Reggiano Cheese,198.735849
Eggplant,80.037736
Zucchini,208.556604
Sun-dried Tomatoes,160.556604


## Cargamos los datos

Vamos a proceder a cargar la prediccion en un csv

In [47]:
def cargar_predicciones(df_prediccion):
    # pasamos el dataframe a un csv
    df_prediccion.to_csv("predicciones.csv")

cargar_predicciones(df_prediccion)